In [8]:
import pandas as pd
from lxml import html
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import random

In [50]:
def parler_parse(dataframe):
    '''
    function to take in the dataframe and parse out the elements we want from the html here
    
    ---------------
    arguments:
        dataframe: pd.DataFrame
            the dataframe we're working with here of the filenames (index) and the html for each file
            
    ---------------
    returns:
        null -> it just operates in place
    '''
    
    # making the tree structure from the html we're using here
    tree = html.fromstring(dataframe["html"]) 
    
    
    # would seem to need some processing -> remove new lines I'd say
    
    # who echoed it (fixed)
    dataframe["echoed_by"] = tree.xpath('//div[contains(@class, "eb--col eb--statement")]//span[contains(@class, "reblock")]/text()')

    # when it was echoed (fixed)
    dataframe["when_echoed"] = tree.xpath('//div[contains(@class, "eb--col eb--timestamp")]//span[contains(@class, "reblock")]/text()')
    
    # timestamp (fixed)
    dataframe["timestamp"] = tree.xpath('//span[contains(@class, "post--timestamp")]/text()')

    # author (fixed)
    dataframe["author"] = tree.xpath('//span[contains(@class, "author--username")]/text()')

    # I need to add a check for if there is no text
    # post content -> confirm works with single p tag -> we'll have to figure out how it handles more though
    dataframe["content"] = tree.xpath('//div[contains(@class, "card--body")]//p/text()')

    # impressions (fixed)
    dataframe["impressions"] = tree.xpath('//span[contains(@class, "impressions--count")]/text()')

    # post comments (fixed)
    dataframe["comments"] = tree.xpath('//span[preceding-sibling::img[contains(@alt, "Post Comments")]]/text()')

    # post echoes (fixed)
    dataframe["echoes"] = tree.xpath('//span[preceding-sibling::img[contains(@alt, "Post Echoes")]]/text()')

    # post upvotes (fixed)
    dataframe["upvotes"] = tree.xpath('//span[preceding-sibling::img[contains(@alt, "Post Upvotes")]]/text()')
    
    return dataframe

In [5]:
tgtwd = 'C:\\Users\\ashbu\\Desktop\\Personal Projects\\Parler Data Merge\\'

In [6]:
post_path = tgtwd +  "\parler_2020-01-06_posts-partial\posts"

In [9]:
%%time
rand_samp_1 = random.choices(os.listdir(post_path), k=1000)

CPU times: total: 2.3 s
Wall time: 2.34 s


In [10]:
rand_files_1 = {file: Path(post_path+"\\"+file).read_text(encoding="utf-8") for file in rand_samp_1}

In [12]:
rand_df_1 = pd.DataFrame.from_dict(rand_files_1, orient="index")

In [13]:
rand_df_1 = rand_df_1.rename(columns={0:"html"})

In [51]:
%%time
test = rand_df_1.apply(parler_parse, axis=1)

CPU times: total: 5.48 s
Wall time: 5.79 s
